In [ ]:
import pandas as pd
from funcy import rpartial
import json
from functools import partial
import functools, re, operator
import numpy as np
import pickle

In [1]:
df = pd.read_csv('./candidate_info_202106020916.csv')
df.dropna().reset_index()

def getData(data):
    
        #Loading data to json
        more_data = json.loads(data)
        
        #Iterating throgh each job dictionory and making it a single string
        current_job = ['{0} {1} {2} {3}'.format(job.get('title').lower(), job.get('company').lower(), job.get('location').lower(),  job.get('dateRange').lower()) for job in more_data.get('workExperience', [])]  
        
        #Iterating throgh each education(school) dictionory and making it a single string 
        curr_school = ['{0} {1} {2} {3}'.format(school.get('degree').lower(), school.get('university').lower(), school.get('location').lower(),  school.get('dateRange').lower()) for school in more_data.get('education', [])]  
        
        #creating text data using name, location, current_job, additionalInformation, summary, schools
        text = '{0} {1} work experience {2}. '.format( more_data['fullName'], more_data['location'] , ' '.join(current_job)) + more_data.get('additionalInformation', '').strip().replace('\n', '').replace('\t', '').lower() + more_data.get('summary', '').strip().replace('\n', '').replace('\t','').lower() + ". Education {}".format(' '.join(curr_school))      
        text = re.sub('[()]', '', text)
        #loading each jobs from workExperience and saving title, compony, hiredate(dateRange) as list of dictionory 
        jobs = [{'title':re.sub('[()]','',job.get('title', '').lower()),'company':re.sub('[()]','',job.get('company', '').lower()), 'hiredate':re.sub('[()]','',job.get('dateRange', '').lower())} for job in more_data.get('workExperience', [])]
        
        #loading each shools from Education and saving degree, university, grad_date(dateRange) as list of dictionory
        schools = [{'degree': re.sub('[()]','',school.get('degree').lower()),'university': re.sub('[()]','',school.get('university').lower()), 'grad_date':re.sub('[()]','',school.get('dateRange').lower())} for school in more_data.get('education', [])]
        
        #Returning all the compoined data {id, location, name, text , jobs, school}
        return {'id': more_data['id'],'location':re.sub('[()]','',more_data['location'].lower()), 'name': re.sub('[()]','',more_data['fullName'].lower()) , 'text':text.lower(), 'jobs':jobs, 'schools': schools}

    

<IPython.core.display.Javascript object>

In [3]:
#Removing more data column which has float values(not json format, bad data)
df = df[df['more_data'].apply(lambda x: not isinstance(x, float))].reset_index()

In [4]:
# calling getData method for ecah more_data
df['more_data_as_p'] = df['more_data'].apply(lambda x: getData(x))

In [5]:
#class for getting training Data or annotated data 
class SpacyAnnotation:
    def __init__(self, candidate_json):
        self.candidate_json = candidate_json 
        self.text  =candidate_json["text"] 
        self.apply_pattern = rpartial(re.search, self.text)
        self.end = 0
        
    def _get_ann(self, pattern, name):
        
        text = self.text[self.end:]
        
        match_obj = re.search(pattern, text)
        if match_obj and match_obj.group():
            
        
            start_  = match_obj.start() + self.end
            end_  = match_obj.end() + self.end
        
            self.end += match_obj.end()
        
            return (start_ ,end_, name)
        return (0,0, name)
        
    def _ann_list(self,cls_name,json_obj): 
        
        ann_list = []
        self.json_obj = json_obj
        for name, p in json_obj.items():
            label = f"{cls_name}_{name.upper()}"
            if name == "company":
                pattern = p.split("  ")
                print(pattern)
                p = pattern[0]
            ann = self._get_ann(p, label)
            ann_list.append(ann)
        return ann_list
    
    @property
    def location(self):
        pattern = self.candidate_json["location"]
        return self._get_ann(pattern , "LOC")
    
    @property
    def name(self):
        pattern = self.candidate_json["name"]
        return self._get_ann(pattern , "NAME")
    
    @property
    def jobs(self):
        jobs =  [ann for job in self.candidate_json["jobs"]
                for ann in self._ann_list( "JOB",job)]
        return jobs
        
    @property
    def edu(self):
        edu = [ ann for school in self.candidate_json["schools"]
              for ann in self._ann_list( "EDU",school)]
        return edu
    
    def get_annotations(self):
        # please don't change the order

        annotations = [ self.name , self.location ] + self.jobs + self.edu  
        return ( self.candidate_json["id"] , self.text , annotations )

In [6]:
#creating SpacyAnnotation object for each data and storing as list of object
spacy_annotation_object_list = [SpacyAnnotation(x) for x in df['more_data_as_p']]

#Creating a refference for get_annotation() method of each object present in the list
data = []
for spacy_annotation_object in spacy_annotation_object_list:
    data.append(spacy_annotation_object.get_annotations())
#     data.append(temp)


['vanisb technology']
['nfc solutions']
['nfc solutions']
['managni systems pvt ltd']
['tecsec technologies private limited']
['tecsec technologies private limited']
['python']
['python']
['']
['']
['3embed software technology and private limited']
['bias infotech pvt ltd']
['novelvox software pvt lmt']
['newgen software technologies limited']
['infozech software pvt. ltd']
['veniteck solutions']
['veniteck solutions']
['visionsoft gbs from']
['javerian services private limited']
['']
['northern trust']
['infosys limited']
['northern trust']
['wipro hr services pvt ltd']
['urban company']
['crossbow labs']
['lexes technology, benagluru']
['zerozilla technology, benagluru']
['techciti technologies private limited']
['shiash info solutions private limited']
['irohub infotech']
['job portal']
['blog app']
['tcs']
['trejhara solutions ltd']
['aurionpro solutions inc']
['aurionpro solutions ltd']
['acg india']
['integrated developer']
['']
['python/django developer job vacanice in any compn

['freelance']
['next level business services']
['net2source inc']
['net2source inc']
['mamsys consulting services ltd']
['nocodez']
['redeem systems pvt ltd']
['itw travel & leisure pvt ltd']
['valtrove people solution pvt ltd']
['wipro technologies']
['bias infotech pvt ltd']
['i-exceed technologies']
['tata consultancy services tcs for 4']
['mentee education']
['mixture lab it solution']
['wipro limited']
['skymongers technologies']
['daystar it academy pvt ltd']
['niit']
['integra micro systems']
['swaragh technologies']
['web developer']
['jspiders training institute']
['techmahindra pvt ltd']
['techmahindra pvt ltd']
['techmahindra pvt ltd']
['snipe tech pvt ltd']
['tata consultancy services']
['tata consultancy services']
['tata consultancy services']
['tata consultancy services']
['tata consultancy services']
['unitypoint health']
['independent health']
['hartford insurance']
['citadel inc']
['krid v solutions']
['studio 11']
['']
['jspider']
['online face recognition attendance

['cisco india pvt. ltd']
['mahindra ltd']
['wipro technologies ltd']
['global logic pvt. ltd']
['persistent system ltd']
['accenture']
['accenture']
['nse.it']
['accenture']
['tcs']
['trejhara solutions ltd']
['aurionpro solutions inc']
['aurionpro solutions ltd']
['tata consultancy services']
['sprint communications']
['efiling of income tax, government of india']
['']
['nib ii bsnl']
['capgemini']
['kpmg']
['sas india']
['tcs']
['capgemini']
['ericsson india ltd']
['aditya birla health insurance']
['majesco']
['unisys - airline products and solution group']
['mph hp company']
['niit technologies']
['kale consultants - airline products and solution group']
['ibs software service']
['kale consultants ltd']
['jvi travels ltd']
['reservation & ticketing']
['capita technology services pvt ltd']
['lti ltd']
['capgemini i pvt ltd']
['wns global services']
['sbi life insurance ltd']
['nirmal datacomm pvt. ltd']
['sify technologies ltd']
['pooja consultancy services pcs']
['rolta india limite

['tata consultancy services']
['tata consultancy services']
['tata consultancy services']
['tata consultancy services']
['hp']
['krystal streem software pvt. ltd']
['wms solution pvt ltd., dubai/noida']
['verizon data service india']
['excelacom technologies private limited']
['low value payments']
['bfs']
['finacle']
['business solutions']
['oracle financial services']
['flexcube for a acc bank project in dublin']
['l&t infotech']
['l&t infotech']
['sales engineering& cloud practice']
['nournet communications']
['hp enterprises']
['unix sme']
['echo mail inc']
['electronics & telecommunications']
['anna university']
['l&t infotech']
['arihant computers']
['electronics and telecommunications engineering']
['infosys']
['cgi india pvt.ltd']
['scii']
['bnp paribas india solutions pvt. ltd australia and new-zealand']
['bnp paribas india solutions pvt. ltd uk']
['bank of america continuum india pvt. ltd']
['bank of america continuum india pvt. ltd']
['bank of america continuum india pvt. lt

['mikelegal, gurugram']
['educo international india pvt. ltd']
['educosoft']
['cliffex software solutions']
['pracify']
['ymgrad']
['chandramohan']
['rebuiltindia']
['nexgen']
['']
['jmoon labs']
['calqlum internet']
['sopra steria india pvt ltd']
['sopra steria india pvt ltd']
['sopra steria india pvt ltd']
['quantum it innovation']
['istrat software pvt. ltd']
['ace overseas education']
['make me builder']
['3embed software technology and private limited']
['madhulata infotech pvt. ltd.']
['nav shri art and culture organisation']
['elephanttree technologies pvt ltd']
['kinetic taigene electrical company pvt. ltd']
['swifterz creative services']
['tcs']
['tata consultancy services tcs']
['northern trust']
['infosys limited']
['northern trust']
['wisewolf solutions pvt. ltd.']
['grassroots bpo pvt.ltd']
['tensorlabs']
['data template info tech private limited']
['fugenx technologies']
['fugenx technologies']
['teknuance info solutions']
['zoho corporation']
['job portal']
['blog app']


error: multiple repeat at position 2

In [7]:
data[0]

('EeubTj7ZYdOs5qkr3ZvMWg',
 "tasaduq khurshid okhla, delhi, delhi work experience java developer vanisb technology  present. myself er tasaduq. completed my bachelors from university of kashmir b.tech cse. education bachelor's university of kashmir north campus dellina baramulla  september 2015 to september 2019",
 [(0, 16, 'NAME'),
  (17, 36, 'LOC'),
  (53, 67, 'JOB_TITLE'),
  (68, 85, 'JOB_COMPANY'),
  (87, 94, 'JOB_HIREDATE'),
  (187, 197, 'EDU_DEGREE'),
  (198, 250, 'EDU_UNIVERSITY'),
  (252, 284, 'EDU_GRAD_DATE')])

In [8]:
data[0][1][17:36]

'okhla, delhi, delhi'

In [75]:
with open('data.pickle', "wb") as fout:
    pickle.dump(data, fout)

In [76]:
a = pickle.load(open('data.pickle','rb'))

In [55]:
a

('EeubTj7ZYdOs5qkr3ZvMWg',
 "tasaduq khurshid okhla, delhi, delhi work experience java developer vanisb technology  present. myself er tasaduq. completed my bachelors from university of kashmir (b.tech cse). education bachelor's university of kashmir north campus dellina baramulla  september 2015 to september 2019",
 [(0, 16, 'NAME'),
  (17, 36, 'LOC'),
  (53, 67, 'JOB_TITLE'),
  (68, 85, 'JOB_COMPANY'),
  (87, 94, 'JOB_HIREDATE'),
  (189, 199, 'EDU_DEGREE'),
  (200, 252, 'EDU_UNIVERSITY'),
  (254, 286, 'EDU_GRAD_DATE')])